## PySpark Partitioning

PySpark partition is a way to split a large dataset into smaller datasets based on one or more partition keys. When you create a DataFrame from a file/table, based on certain parameters PySpark creates the DataFrame with a certain number of partitions in memory. This is one of the main advantages of PySpark DataFrame over Pandas DataFrame. Transformations on partitioned data run faster as they execute transformations parallelly for each partition.

PySpark supports partition in two ways; partition in memory (DataFrame) and partition on the disk (File system).

**Partition in memory**: You can partition or repartition the DataFrame by calling `repartition()` or `coalesce()` transformations.

**Partition on disk**: While writing the PySpark DataFrame back to disk, you can choose how to partition the data based on columns using `partitionBy()`.

##### Partition Advantages

As you are aware PySpark is designed to process large datasets with 100x faster than the tradition processing, this wouldn’t have been possible with out partition. Below are some of the advantages using PySpark partitions on memory or on disk.  

* Fast accessed to the data
* Provides the ability to perform an operation on a smaller dataset

In [0]:
dbutils.library.restartPython() # Removes Python state, but some libraries might not work without calling this command.dbutils.restartPython()

#### Load libraries

In [0]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import IntegerType, DateType, StringType, StructType, StructField, ArrayType, MapType, DoubleType
from pyspark.sql.functions import lit, col, expr, when, sum, avg, max, min, mean, count, udf, explode, concat_ws

#### Create Spark session

In [0]:
spark = SparkSession.builder.appName('PySpark Partitions').getOrCreate()

#### Create Dataframe

In [0]:
from datetime import datetime

data = [
  ('item1', 5, datetime.strptime('2021-06-15','%Y-%m-%d')),
  ('item2', 1, datetime.strptime('2021-06-20','%Y-%m-%d')),
  ('item8', 9, datetime.strptime('2021-06-20','%Y-%m-%d')),
  ('item3', 2, datetime.strptime('2021-06-20','%Y-%m-%d')),
  ('item1', 3, datetime.strptime('2021-07-05','%Y-%m-%d')),
  ('item3', 4, datetime.strptime('2021-07-25','%Y-%m-%d')),
  ('item2', 1, datetime.strptime('2021-07-30','%Y-%m-%d')),
  ('item4', 6, datetime.strptime('2021-08-01','%Y-%m-%d')),
  ('item2', 8, datetime.strptime('2021-08-01','%Y-%m-%d')),
  ('item5', 8, datetime.strptime('2021-08-03','%Y-%m-%d'))
]

schema = StructType([
  StructField('item', StringType(), True),
  StructField('quantity', IntegerType(), True),
  StructField('date', DateType(), True)
])

df = spark.createDataFrame(data=data, schema=schema)
df.printSchema()
df.show()

root
-- item: string (nullable = true)
-- quantity: integer (nullable = true)
-- date: date (nullable = true)

+-----+--------+----------+
 item|quantity| date|
+-----+--------+----------+
item1| 5|2021-06-15|
item2| 1|2021-06-20|
item8| 9|2021-06-20|
item3| 2|2021-06-20|
item1| 3|2021-07-05|
item3| 4|2021-07-25|
item2| 1|2021-07-30|
item4| 6|2021-08-01|
item2| 8|2021-08-01|
item5| 8|2021-08-03|
+-----+--------+----------+

In [0]:
table_name = 'temp.partitions_testing'
table_path = f'/mnt/{table_name}'

#### partitionBy()

In [0]:
(df
.write
.format('delta')
.mode('overwrite')
.partitionBy('date')
.save(table_path))

In [0]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing

total 40K
drwxrwxrwx 2 root root 4.0K Aug 4 09:08 .
drwxrwxrwx 2 root root 4.0K Aug 4 09:08 ..
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 _delta_log
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 date=2021-06-15
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 date=2021-06-20
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 date=2021-07-05
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 date=2021-07-25
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 date=2021-07-30
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 date=2021-08-01
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 date=2021-08-03

In [0]:
#spark.sql(f"create table if not exists {table_name} using delta location '{table_path}'")

In [0]:
#spark.sql(f"drop table if exists {table_name}")

In [0]:
%sh
rm -rf /dbfs/mnt/temp.partitions_testing

#### partitionBy() Multiple Columns

In [0]:
(df
.write
.format('delta')
.mode('overwrite')
.partitionBy('date','item')
.save(table_path))

In [0]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing/date=2021-06-20

total 20K
drwxrwxrwx 2 root root 4.0K Aug 4 09:08 .
drwxrwxrwx 2 root root 4.0K Aug 4 09:08 ..
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 item=item2
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 item=item3
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 item=item8

In [0]:
%sh
rm -rf /dbfs/mnt/temp.partitions_testing

#### Using repartition() and partitionBy() together

For each partition column, if you wanted to further divide into several partitions, use `repartition()` and `partitionBy()`

In [0]:
(df
.repartition(2)
.write
.format('delta')
.mode('overwrite')
.partitionBy('date')
.save(table_path))

In [0]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing/date=2021-06-20

total 10K
drwxrwxrwx 2 root root 4.0K Aug 4 09:08 .
drwxrwxrwx 2 root root 4.0K Aug 4 09:08 ..
-rwxrwxrwx 1 root root 712 Aug 4 09:08 part-00000-3e2647f5-130d-4ab4-88c6-1a5c878ba7e8.c000.snappy.parquet
-rwxrwxrwx 1 root root 725 Aug 4 09:08 part-00001-d3ea9133-2829-4b0b-aa82-c5aba564424e.c000.snappy.parquet

In [0]:
%sh
rm -rf /dbfs/mnt/temp.partitions_testing

#### Control Number of Records per Partition File

In [0]:
# creates multiple part files for each date and each part file contains just 2 records.
(df
.write
.format('delta')
.mode('overwrite')
.option('maxRecordsPerFile', 2)
.partitionBy('date')
.save(table_path))

In [0]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing/date=2021-06-20

total 11K
drwxrwxrwx 2 root root 4.0K Aug 4 09:08 .
drwxrwxrwx 2 root root 4.0K Aug 4 09:08 ..
-rwxrwxrwx 1 root root 725 Aug 4 09:08 part-00001-440e88db-718b-4220-b020-31bfd6374fa2.c000.snappy.parquet
-rwxrwxrwx 1 root root 725 Aug 4 09:08 part-00002-5684ff65-43b5-4865-94aa-789570fb08e7.c000.snappy.parquet
-rwxrwxrwx 1 root root 725 Aug 4 09:08 part-00003-59f790d3-d80b-40e6-b899-3fdb936dbd74.c000.snappy.parquet

In [0]:
%sh
rm -rf /dbfs/mnt/temp.partitions_testing/

#### Read a Specific Partition

In [0]:
(df
.write
.format('delta')
.mode('overwrite')
.partitionBy('date','item')
.save(table_path))

In [0]:
%sh
ls -lah /dbfs/mnt/temp.partitions_testing/date=2021-06-20

total 20K
drwxrwxrwx 2 root root 4.0K Aug 4 09:08 .
drwxrwxrwx 2 root root 4.0K Aug 4 09:08 ..
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 item=item2
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 item=item3
drwxrwxrwx 2 root root 4.0K Jul 30 13:25 item=item8

In [0]:
partition_path = f'{table_path}/date=2021-06-20'

dfSinglePart = spark.read.format('delta').load(partition_path)
dfSinglePart.printSchema()
dfSinglePart.show()

root
-- item: string (nullable = true)
-- quantity: integer (nullable = true)
-- date: date (nullable = true)

+-----+--------+----------+
 item|quantity| date|
+-----+--------+----------+
item3| 2|2021-06-20|
item8| 9|2021-06-20|
item2| 1|2021-06-20|
+-----+--------+----------+

In [0]:
%sh
rm -rf /dbfs/mnt/temp.partitions_testing/

#### The end of the notebook